In [1]:
import pandas as pd
import numpy as np
from scipy import sparse as ssp
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
import xgboost as xgb
# from generate_selftrained_w2v import gen_w2v
# from config import path

path = '../cache/'
seed = 1024
np.random.seed(seed)
# path = '../input/'


In [2]:
feats= ['comment_text_unigram','comment_text_bigram']
X = []
for f in feats:
    X.append(pd.read_pickle(path+'train_%s_tfidf_nmf.pkl'%f))
    X.append(pd.read_pickle(path+'train_%s_tfidf_svd.pkl'%f))
# X.append(np.loadtxt(path+'train_spacy_diff_pretrained.txt'))


In [3]:
feats3= [
'comment_text',
]
for f in feats3:
    X.append(pd.read_pickle(path+'train_%s_lda.pkl'%f))

In [4]:
X = np.hstack(X)#.tocsr()
len_train = X.shape[0]

In [5]:
train_unigram_features = pd.read_csv(path+'train_unigram_features.csv').values
train_bigram_features = pd.read_csv(path+'train_bigram_features.csv').values

In [6]:
train_porter_stop_cuss_features = pd.read_csv(path+'train_porter_stop_cuss_features.csv').values

In [7]:
train_idf_stats_features = pd.read_csv(path+'train_idf_stats_features.csv').values

In [8]:
#train_pattern = pd.read_pickle(path+'train.pattern.pkl').reshape((X.shape[0],3))
train_pattern = pd.read_pickle(path+'train.pattern.pkl').reshape((X.shape[0],1))

In [9]:
print(train_pattern.shape)
print(X.shape)

(159571, 1)
(159571, 50)


In [10]:
train = pd.read_csv('../data/train.csv')
train_count_ct_mark_all = train['comment_text'].astype(str).apply( lambda x : x.count("!")+x.count("?")+x.count(",")+x.count(".")+x.count("*")+x.count("@")+x.count("#")+x.count("$")+x.count("%")+x.count("^")+x.count("&")).values.reshape(-1,1)
train_count_ct_mark_bang = train['comment_text'].astype(str).apply( lambda x : x.count("!")).values.reshape(-1,1)
train_count_ct_mark_star = train['comment_text'].astype(str).apply( lambda x : x.count("*")).values.reshape(-1,1)
train_count_ct_mark_special = train['comment_text'].astype(str).apply( lambda x : x.count("!")+x.count("@")+x.count("#")+x.count("$")+x.count("%")+x.count("^")+x.count("&")+x.count("*")).values.reshape(-1,1)

train_ratio_bang_mark = (train_count_ct_mark_bang)/(train_count_ct_mark_all+1e-7)
train_ratio_star_mark = (train_count_ct_mark_star)/(train_count_ct_mark_all+1e-7)
train_ratio_special_mark = (train_count_ct_mark_special)/(train_count_ct_mark_all+1e-7)



In [11]:
train_char_length = train['comment_text'].apply(lambda x: len(str(x))).reshape(-1,1)
train_num_words = train['comment_text'].map(lambda x: len(str(x).split(' '))).reshape(-1,1)


/home/watts/anaconda3/envs/ktc/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/home/watts/anaconda3/envs/ktc/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [12]:
train_entropy_unigram = pd.read_csv(path+'train_entropy_unigram.csv').values
train_entropy_bigram = pd.read_csv(path+'train_entropy_bigram.csv').values

In [13]:
print(train_entropy_bigram.shape)
print(train_entropy_unigram.shape)

(159571, 2)
(159571, 2)


In [14]:
print(train_unigram_features.shape)
print(train_bigram_features.shape)
print(train_porter_stop_cuss_features.shape)

print(train_idf_stats_features.shape)

print(train_pattern.shape)

print(train_count_ct_mark_all.shape)
print(train_count_ct_mark_bang.shape)
print(train_count_ct_mark_star.shape)
print(train_count_ct_mark_special.shape)
print(train_ratio_bang_mark.shape)
print(train_ratio_star_mark.shape)
print(train_ratio_special_mark.shape)
print(train_char_length.shape)
print(train_num_words.shape)
print(train_entropy_unigram.shape)
print(train_entropy_bigram.shape)

(159571, 5)
(159571, 5)
(159571, 4)
(159571, 5)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 1)
(159571, 2)
(159571, 2)


In [15]:
train_basic_features = np.hstack([
    train_unigram_features,
    train_bigram_features,
    train_porter_stop_cuss_features,
    
    train_idf_stats_features,
    
    train_pattern,
    
    train_count_ct_mark_all,
    train_count_ct_mark_bang,
    train_count_ct_mark_star,
    train_count_ct_mark_special,
    train_ratio_bang_mark,
    train_ratio_star_mark,
    train_ratio_special_mark,
    train_char_length,
    train_num_words,
    train_entropy_unigram,
    train_entropy_bigram
    
    ])


In [16]:
X = np.hstack([X,train_basic_features])#.tocsr()

In [17]:
np.save('../cache/X', X)

In [18]:
train_df = pd.read_csv("../data/train.csv")

In [19]:
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [20]:
y1 = train_df['toxic'].values
y2 = train_df['severe_toxic'].values
y3 = train_df['obscene'].values
y4 = train_df['threat'].values
y5 = train_df['insult'].values
y6 = train_df['identity_hate'].values
y7 = train_df.loc[:,['toxic','severe_toxic','obscene','threat','insult','identity_hate']]


In [33]:
np.save('../cache/y1', y1)
np.save('../cache/y2', y2)
np.save('../cache/y3', y3)
np.save('../cache/y4', y4)
np.save('../cache/y5', y5)
np.save('../cache/y6', y6)
np.save('../cache/y7', y7)

In [21]:
del train_unigram_features
del train_bigram_features
del train_porter_stop_cuss_features

del train_idf_stats_features

del train_count_ct_mark_all
del train_count_ct_mark_bang
del train_count_ct_mark_star
del train_count_ct_mark_special
del train_ratio_bang_mark
del train_ratio_star_mark
del train_ratio_special_mark
del train_char_length
del train_num_words
del train_entropy_unigram
del train_entropy_bigram


In [22]:
print(X.shape)

(159571, 83)


In [22]:
153164-152351

813

In [23]:
feats= ['comment_text_unigram','comment_text_bigram']
X_t = []
for f in feats:
    X_t.append(pd.read_pickle(path+'test_%s_tfidf_nmf.pkl'%f))
    X_t.append(pd.read_pickle(path+'test_%s_tfidf_svd.pkl'%f))

# feats2= [
#     'comment_text',
#     'comment_text_porter',
# ]
# for f in feats2:
#     X_t.append(pd.read_pickle(path+'test_%s_nmf.pkl'%f))
#     X_t.append(pd.read_pickle(path+'test_%s_svd.pkl'%f))

feats3= [
    'comment_text',
]
for f in feats3:
    X_t.append(pd.read_pickle(path+'test_%s_lda.pkl'%f))

for l in X_t:
    print(l.shape)
X_t = np.hstack(X_t)#.tocsr()
len_test = X_t.shape[0]


(153164, 4)
(153164, 16)
(153164, 4)
(153164, 16)
(153164, 10)


In [24]:
test_unigram_features = pd.read_csv(path+'test_unigram_features.csv').values
test_bigram_features = pd.read_csv(path+'test_bigram_features.csv').values
test_porter_stop_cuss_features = pd.read_csv(path+'test_porter_stop_cuss_features.csv').values

test_idf_stats_features = pd.read_csv(path+'test_idf_stats_features.csv').values
test_pattern = pd.read_pickle(path+'test.pattern.pkl').reshape((X_t.shape[0],1))

In [25]:
test = pd.read_csv('../data/test.csv')
test_count_ct_mark_all = test['comment_text'].astype(str).apply( lambda x : x.count("!")+x.count("?")+x.count(",")+x.count(".")+x.count("*")+x.count("@")+x.count("#")+x.count("$")+x.count("%")+x.count("^")+x.count("&")).values.reshape(-1,1)
test_count_ct_mark_bang = test['comment_text'].astype(str).apply( lambda x : x.count("!")).values.reshape(-1,1)
test_count_ct_mark_star = test['comment_text'].astype(str).apply( lambda x : x.count("*")).values.reshape(-1,1)
test_count_ct_mark_special = test['comment_text'].astype(str).apply( lambda x : x.count("!")+x.count("@")+x.count("#")+x.count("$")+x.count("%")+x.count("^")+x.count("&")+x.count("*")).values.reshape(-1,1)

test_ratio_bang_mark = (test_count_ct_mark_bang)/(test_count_ct_mark_all+1e-7)
test_ratio_star_mark = (test_count_ct_mark_star)/(test_count_ct_mark_all+1e-7)
test_ratio_special_mark = (test_count_ct_mark_special)/(test_count_ct_mark_all+1e-7)

test_char_length = test['comment_text'].apply(lambda x: len(str(x))).values.reshape(-1,1)
test_num_words = test['comment_text'].map(lambda x: len(str(x).split(' '))).values.reshape(-1,1)

test_entropy_unigram = pd.read_csv(path+'test_entropy_unigram.csv')
test_entropy_bigram = pd.read_csv(path+'test_entropy_bigram.csv')

In [26]:
print(test_unigram_features.shape)
print(test_bigram_features.shape)
print(test_porter_stop_cuss_features.shape)

print(test_idf_stats_features.shape)

print(test_pattern.shape)

print(test_count_ct_mark_all.shape)
print(test_count_ct_mark_bang.shape)
print(test_count_ct_mark_star.shape)
print(test_count_ct_mark_special.shape)
print(test_ratio_bang_mark.shape)
print(test_ratio_star_mark.shape)
print(test_ratio_special_mark.shape)
print(test_char_length.shape)
print(test_num_words.shape)
print(test_entropy_unigram.shape)
print(test_entropy_bigram.shape)

(153164, 5)
(153164, 5)
(153164, 4)
(153164, 5)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 1)
(153164, 2)
(153164, 2)


In [27]:
test_basic_features = np.hstack([
    test_unigram_features,
    test_bigram_features,
    test_porter_stop_cuss_features,
    
    test_idf_stats_features,
    
    test_pattern,
    
    test_count_ct_mark_all,
    test_count_ct_mark_bang,
    test_count_ct_mark_star,
    test_count_ct_mark_special,
    test_ratio_bang_mark,
    test_ratio_star_mark,
    test_ratio_special_mark,
    test_char_length,
    test_num_words,
    test_entropy_unigram,
    test_entropy_bigram,
    
    ])


In [28]:
X_t = np.hstack([X_t,test_basic_features])#.tocsr()

In [29]:
np.save('../cache/X_t', X_t)

In [30]:
del test_unigram_features
del test_bigram_features
del test_porter_stop_cuss_features

del test_idf_stats_features

del test_count_ct_mark_all
del test_count_ct_mark_bang
del test_count_ct_mark_star
del test_count_ct_mark_special
del test_ratio_bang_mark
del test_ratio_star_mark
del test_ratio_special_mark
del test_char_length
del test_num_words
del test_entropy_unigram
del test_entropy_bigram

In [31]:
print(X_t.shape)

(153164, 83)


In [ ]:
# from utils import make_mf_classification

# X = pd.DataFrame(X).replace(np.nan,0).values
# X_t = pd.DataFrame(X_t).replace(np.nan,0).values
# X = pd.DataFrame(X).replace(np.inf,0).values
# X_t = pd.DataFrame(X_t).replace(np.inf,0).values


# import lightgbm as lgb
# clf = lgb.LGBMClassifier(
#                         num_leaves=31,
#                         learning_rate= 0.008,
#                         n_estimators=26972,
#                         subsample = 0.75,
#                         colsample_bytree = 0.54,
#                         min_child_weight=13,
#                         min_split_gain=0.3,
#                         )

# make_mf_classification(X ,y, clf, X_t, n_folds=5,seed=seed,nb_epoch=1,max_features=1.0,name='lgb_3',path=path)



In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(
#             n_estimators=500,
#             max_depth = 18,
#             min_samples_split=16,
#             bootstrap=False,
#             max_features=0.1,
#             n_jobs=8,
#             random_state=seed,
#             verbose=True,
#             )

# make_mf_classification(X ,y, clf, X_t, n_folds=5,seed=seed,nb_epoch=1,max_features=1.0,name='rf_3',path=path)



# from sklearn.ensemble import ExtraTreesClassifier
# clf = ExtraTreesClassifier(
#             n_estimators=500,
#             max_depth = 23,
#             min_samples_split=8,
#             bootstrap=False,
#             max_features=0.3,
#             n_jobs=8,
#             random_state=seed,
#             verbose=True,
#             )

# make_mf_classification(X ,y, clf, X_t, n_folds=5,seed=seed,nb_epoch=1,max_features=1.0,name='et_3',path=path)



In [ ]:
# clf = xgb.XGBClassifier(
#         n_estimators=9326/3,
#         learning_rate = 0.093/3,
#         max_depth=8,
#         subsample=0.75,
#         colsample_bytree = 0.54,
#         gamma = 0.3,
#         reg_lambda=0,
#         min_child_weight=13,
#         seed = seed,
#         )
# make_mf_classification(X ,y, clf, X_t, n_folds=5,seed=seed,nb_epoch=1,max_features=1.0,name='xgb_3',path=path)




In [ ]:
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint,Callback
from keras.layers.noise import GaussianNoise
from keras import backend as K
from keras.layers import Input, Embedding, LSTM, Dense,Flatten, Dropout, merge,Convolution1D,MaxPooling1D,Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD,Nadam
from keras.layers.advanced_activations import PReLU,LeakyReLU,ELU
from keras.models import Model
from keras.utils.visualize_util import plot
from keras.utils.np_utils import to_categorical
from sklearn.base import BaseEstimator

def build_model(maxlen=238,hidden=512):
    inputs = []
    inputs_q1 = Input(shape=(maxlen,),name='input_q1',sparse=False)
    inputs.append(inputs_q1)

    fc1 = Dense(hidden)(inputs_q1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.2)(fc1)

    fc1 = Dense(hidden)(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.2)(fc1)

    fc1 = Dense(hidden)(fc1)
    fc1 = PReLU()(fc1)
    fc1 = BatchNormalization()(fc1)
    fc1 = Dropout(0.2)(fc1)

    
    outputs = Dense(1,activation='sigmoid')(fc1)
    model = Model(input=inputs, output=outputs)

    model.compile(
                optimizer='adam',
                loss = 'binary_crossentropy',
              )
    return model

class NN(BaseEstimator):
    def __init__(self,input_shape,hidden=128):
        self.input_shape = input_shape
        self.hidden = hidden
    def fit(self,X,y):
        model = build_model(maxlen=self.input_shape,hidden=self.hidden)
        model.fit(X,y,epochs=250,batch_size=128,shuffle=True,verbose=1)
        self.model=model
    def predict_proba(self,X):
        y_p = self.model.predict(X,batch_size=1024).ravel()
        y_p_inv= np.ones(y_p.shape[0])-y_p
        y_p = np.hstack([y_p_inv.reshape(-1,1),y_p.reshape(-1,1)])
        return y_p

scaler = MinMaxScaler()
scaler.fit(X)
clf = NN(X.shape[1],128)
make_mf_classification(scaler.transform(X) ,y, clf, scaler.transform(X_t), n_folds=5,seed=seed,nb_epoch=1,max_features=1.0,name='nn_3',path=path)

